In [2]:
import torch
import torch.nn as nn

import onmt
import onmt.inputters
import onmt.modules
import onmt.utils

In [3]:
!pwd

/Users/aziai/Dropbox/github/cs224n-project/OpenNMT-py


In [4]:
vocab_fields = torch.load("../openmt_geoquery.vocab.pt")

src_text_field = vocab_fields["src"].base_field
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token]

tgt_text_field = vocab_fields['tgt'].base_field
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token]

In [5]:
emb_size = 100
rnn_size = 500
# Specify the core model.

encoder_embeddings = onmt.modules.Embeddings(emb_size, len(src_vocab),
                                             word_padding_idx=src_padding)

encoder = onmt.encoders.RNNEncoder(hidden_size=rnn_size, num_layers=1,
                                   rnn_type="LSTM", bidirectional=True,
                                   embeddings=encoder_embeddings)

decoder_embeddings = onmt.modules.Embeddings(emb_size, len(tgt_vocab),
                                             word_padding_idx=tgt_padding)
decoder = onmt.decoders.decoder.InputFeedRNNDecoder(
    hidden_size=rnn_size, num_layers=1, bidirectional_encoder=True, 
    rnn_type="LSTM", embeddings=decoder_embeddings)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = onmt.models.model.NMTModel(encoder, decoder)
model.to(device)

# Specify the tgt word generator and loss computation module
model.generator = nn.Sequential(
    nn.Linear(rnn_size, len(tgt_vocab)),
    nn.LogSoftmax(dim=-1))

loss = onmt.utils.loss.NMTLossCompute(
    criterion=nn.NLLLoss(ignore_index=tgt_padding, reduction="sum"),
    generator=model.generator)

In [31]:
from itertools import product

In [35]:
params = {
    'x': {1,2},
    'y': {4,5,6}
}


In [44]:
type({1})

set

In [38]:
params.keys()

dict_keys(['x', 'y'])

In [41]:
dict({'1': 2}, **{'2': 3})

{'1': 2, '2': 3}

In [27]:
import subprocess

In [29]:
q = subprocess.check_output(['wc', '-l', 'dev.ipynb'])

In [30]:
q.decode('utf-8')

'     299 dev.ipynb\n'

In [15]:
import uuid

In [26]:
str(uuid.uuid4()).replace('-', '')

'87c5405dce5048ae97df5abd1a53b123'

In [14]:
model.generator

Sequential(
  (0): Linear(in_features=500, out_features=159, bias=True)
  (1): LogSoftmax()
)

In [10]:
lr = 1
torch_optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optim = onmt.utils.optimizers.Optimizer(
    torch_optimizer, learning_rate=lr, max_grad_norm=2)

In [11]:
model.encoder.eval()
model.decoder.train()

InputFeedRNNDecoder(
  (embeddings): Embeddings(
    (make_embedding): Sequential(
      (emb_luts): Elementwise(
        (0): Embedding(159, 100, padding_idx=1)
      )
    )
  )
  (dropout): Dropout(p=0.0)
  (rnn): StackedLSTM(
    (dropout): Dropout(p=0.0)
    (layers): ModuleList(
      (0): LSTMCell(600, 500)
    )
  )
  (attn): GlobalAttention(
    (linear_in): Linear(in_features=500, out_features=500, bias=False)
    (linear_out): Linear(in_features=1000, out_features=500, bias=False)
  )
)

In [12]:
from itertools import chain
train_data_file = "../openmt_geoquery.train.0.pt"
valid_data_file = "../openmt_geoquery.valid.0.pt"
train_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths=[train_data_file],
                                                     fields=vocab_fields,
                                                     batch_size=50,
                                                     batch_size_multiple=1,
                                                     batch_size_fn=None,
                                                     device=device,
                                                     is_train=True,
                                                     repeat=True)

valid_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths=[valid_data_file],
                                                     fields=vocab_fields,
                                                     batch_size=10,
                                                     batch_size_multiple=1,
                                                     batch_size_fn=None,
                                                     device=device,
                                                     is_train=False,
                                                     repeat=False)

In [9]:
report_manager = onmt.utils.ReportMgr(
    report_every=50, start_time=None, tensorboard_writer=None)
trainer = onmt.Trainer(model=model,
                       train_loss=loss,
                       valid_loss=loss,
                       optim=optim,
                       report_manager=report_manager)
trainer.train(train_iter=train_iter,
              train_steps=400,
              valid_iter=valid_iter,
              valid_steps=200)

/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


TypeError: forward() got an unexpected keyword argument 'bptt'